# Load candidates

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
CACHE = '/home/anhphantq/vdb/cache_train'
import os
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import psutil

train_data = []

with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = min(metadata['num_frags'], 10000)
max_session = metadata['max_session']

with tqdm(total= int (psutil.virtual_memory().total  / 1024  / 1024 / 1024), desc='RAM (GB)', position=0, leave= False) as rambar:

  for i in tqdm(range(0, num_frags)):

    train_data.append(pd.read_parquet(CACHE + f'/data_{i}.parquet'))
    rambar.n= int(psutil.virtual_memory().used / 1024  / 1024 / 1024)
    rambar.refresh()




RAM (GB):   0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

In [3]:
train_data = pd.concat(train_data)

In [4]:
candidates = train_data
candidates.shape

(98120900, 159)

In [5]:
import pandas as pd
pd.set_option('display.max_columns', candidates.shape[1])

In [6]:
candidates

,user,item,num_clicks,num_carts,num_orders,item_degree,item_pr,recent_num_clicks,recent_num_carts,recent_num_orders,item_recent_degree,item_recent_pr,recent_day14_type0,recent_day14_type1,recent_day14_type2,recent_day13_type0,recent_day13_type1,recent_day13_type2,recent_day12_type0,recent_day12_type1,recent_day12_type2,recent_day11_type0,recent_day11_type1,recent_day11_type2,recent_day10_type0,recent_day10_type1,recent_day10_type2,recent_day9_type0,recent_day9_type1,recent_day9_type2,recent_day8_type0,recent_day8_type1,recent_day8_type2,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,recent_day4_type0,recent_day4_type1,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2,item_glob_last_action,item_glob_first_action,item_glob_time_decay_sum,item_glob_time_decay_sum_click,item_glob_time_decay_sum_cart,item_glob_time_decay_sum_order,item_glob_item_glob_durability,qou_num_cosub_mean,qou_coclick_sub_coef_mean,qou_cocart_sub_coef_mean,qou_coorder_sub_coef_mean,qou_coclick_time_decay_mean,qou_cocart_time_decay_mean,qou_coorder_time_decay_mean,qou_num_appearance_mean,qou_num_in_k_most_recent_items_mean,qou_num_happend_later_mean,qou_num_happend_before_mean,qou_happend_later_ratio_mean,qou_last_interact_mean,qou_lincom_sub_coef_1_3_10_mean,qou_lincom_time_decay_1_3_10_mean,qou_lincom_sub_coef_1_10_3_mean,qou_lincom_time_decay_1_10_3_mean,qou_lincom_sub_coef_3_1_10_mean,qou_lincom_time_decay_3_1_10_mean,qou_lincom_sub_coef_3_10_1_mean,qou_lincom_time_decay_3_10_1_mean,qou_lincom_sub_coef_10_1_3_mean,qou_lincom_time_decay_10_1_3_mean,qou_lincom_sub_coef_10_3_1_mean,qou_lincom_time_decay_10_3_1_mean,qou_lincom_sub_coef_0.5_10_0.5_mean,qou_lincom_time_decay_0.5_10_0.5_mean,qou_lincom_sub_coef_10_0.5_0.5_mean,qou_lincom_time_decay_10_0.5_0.5_mean,qou_lincom_sub_coef_0.5_0.5_10_mean,qou_lincom_time_decay_0.5_0.5_10_mean,qou_num_cosub_sqrt_num_cousers,qou_coclick_sub_coef_sqrt_num_cousers,qou_cocart_sub_coef_sqrt_num_cousers,qou_coorder_sub_coef_sqrt_num_cousers,qou_coclick_time_decay_sqrt_num_cousers,qou_cocart_time_decay_sqrt_num_cousers,qou_coorder_time_decay_sqrt_num_cousers,qou_num_appearance_sqrt_num_cousers,qou_num_in_k_most_recent_items_sqrt_num_cousers,qou_num_happend_later_sqrt_num_cousers,qou_num_happend_before_sqrt_num_cousers,qou_happend_later_ratio_sqrt_num_cousers,qou_last_interact_sqrt_num_cousers,num_sub,consistency,num_actions,user_degree,user_pr,user_recent_degree,user_recent_pr,cointeractions_coef_clicks_clicks,cointeractions_coef_clicks_carts,cointeractions_coef_clicks_orders,cointeractions_coef_carts_clicks,cointeractions_coef_carts_carts,cointeractions_coef_carts_orders,cointeractions_coef_orders_clicks,cointeractions_coef_orders_carts,cointeractions_coef_orders_orders,cointeractions_time_decay_clicks_clicks,cointeractions_time_decay_clicks_carts,cointeractions_time_decay_clicks_orders,cointeractions_time_decay_carts_clicks,cointeractions_time_decay_carts_carts,cointeractions_time_decay_carts_orders,cointeractions_time_decay_orders_clicks,cointeractions_time_decay_orders_carts,cointeractions_time_decay_orders_orders,fitness,inter_clicks,inter_carts,inter_orders,inter_num_sub,inter_time_decay,inter_lts,inter_fts,inter_durability,inter_num_interacts,is_level1,user_clicks_cnt,user_carts_cnt,user_orders_cnt,user_total_actions_cnt,user_clicks_ratio,user_carts_ratio,user_orders_ratio,user_ss_ts_max,user_ss_ts_min,user_ss_ts_mean,item_item_cnt,item_item_clicks_cnt,item_item_carts_cnt,item_item_orders_cnt,item_item_total_actions_cnt,item_item_clicks_ratio,item_item_carts_ratio,item_item_orders_ratio
0,12036836.0,12910997.0,686.0,120.0,89.0,598.0,2.145596e-06,165.0,41.0,25.0,131.0,9.935047e-07,25.0,5.0,3.0,18.0,5.0,5.0,27.0,9.0,4.0,21.0,5.0,2.0,29.0,8.0,3.0,12.0,2.0,2.0,22.0,4.0,2.0,11.0,3.0,

# Train params

In [9]:
from meow_utils import *

import numba as nb
from numba import prange
@nb.njit(nb.float64(nb.float64[:], nb.float32[:], nb.int64[:]))
def recall20(approxes, target, groups):
    total = 0
    nonempty = 0
    group_ends = np.concatenate((np.ravel(np.argwhere(groups[:-1] != groups[1:])), np.array([len(groups) -1]))) + 1
    group_starts = np.concatenate((np.array([0]), group_ends[:-1]))

    for i in prange(len(group_starts)):
        group_end = group_ends[i]
        group_start = group_starts[i]
        ranks = np.argsort(approxes[group_start:group_end])[::-1]
        hits = 0
        for i in prange(min(len(ranks), 20)):
            hits += target[group_start + ranks[i]]

        actual = min(20, target[group_start:group_end].sum())
        if actual > 0:
            total += hits / actual
            nonempty += 1
    return total / nonempty

class GlobalRecall20(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight = None):
        try:
            groups = weight.astype(int)
        except:
            return 0, 0
            
        
        
        return recall20(approxes[0], target, groups), 0

learning_rate = [0.5, 0.1, 0.1]

PARAMS = {
    # 'learning_rate': 0.5,
    'iterations': 1000,
    'early_stopping_rounds' : 20,
    'max_bin': 256,
    'depth': 7,
    'eval_metric': GlobalRecall20(),
    'use_best_model': True,
    'thread_count': -1,
    'subsample': 1
}
N_SPLITS = 5
NUM_CLICK_SAMPLES = 3
NUM_SAMPLES = 3
MODEL_PATH = '/home/anhphantq/meowmeow'
VERSION = '15'


# Train for each type

In [10]:
from meow_utils import get_len_group

In [11]:
import pandas as pd 
import numpy as np
tar = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
tar = tar.loc[(tar['session'] <= candidates['user'].max()) & (tar['session'] >= candidates['user'].min()) ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session', 'type']].rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['type'] = tar['type'].apply(lambda x: 0 if x == 'clicks' else 1 if x == 'carts' else 2)
tar['label'] = 1
tar['item'] = tar['item'] + max_session
tar['user_type'] = tar['user'] * 10 + tar['type']

In [12]:
labels = []
users = candidates.user.values
items = candidates.item.values
from tqdm.notebook import tqdm
for i in tqdm(range(3)):
  cnt = 0
  tar_small = tar[tar['type'] == i].sort_values(['user', 'item']).set_index(['user', 'item'])
  
  tar_small_index = set(tar_small.index.tolist())
  label = []

  for i in tqdm(range(candidates.shape[0]), leave = False):
    if (users[i], items[i]) in tar_small_index:
      label.append(1)
      cnt += 1
    else: 
      label.append(0)
  labels.append(label)
  print(f'Found {cnt} positives', end = ' ')
del tar, tar_small_index
  



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/98120900 [00:00<?, ?it/s]

Found 753055 positives 

  0%|          | 0/98120900 [00:00<?, ?it/s]

Found 192092 positives 

  0%|          | 0/98120900 [00:00<?, ?it/s]

Found 144328 positives 

In [13]:
labels = np.array(labels)

In [14]:

from sklearn.model_selection import GroupKFold
import os
import matplotlib.pyplot as plt
import shutil
from meow_utils import plot_importance
from catboost import CatBoostRanker, Pool
import seaborn as sns
sns.set()
%matplotlib inline
def train(t, id_type, draw_importance = False):
  global candidates, labels
  name = f'/{t}_only_'
  try:
    shutil.rmtree(MODEL_PATH + name + VERSION)
  except:
    pass
  else:
    print('remove old version')
  fig, axs = plt.subplots(1, N_SPLITS, figsize = (300, 30))
  skf = GroupKFold(n_splits=N_SPLITS)
  
  
  X_all = candidates.iloc[:, 2 :]
  y_all = labels[id_type]
  
  
  print('Building pool...')
  data_all = Pool(X_all, y_all, thread_count = os.cpu_count())
  print('Pool builed!!!!!!')
  for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, labels[id_type], groups=candidates['user'] )):
      neg_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 0))
      pos_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 1))
      #downsample many times
      for sample in range(NUM_SAMPLES if id_type != 0 else NUM_CLICK_SAMPLES):
        print(f'FOLD: {fold} Sample: {sample}')
        n_index = np.random.permutation(neg_index)[:len(pos_index) * 20]
        index = np.union1d(n_index, pos_index)
        
        data_train = data_all.slice(train_idx).slice(index)
        data_train.set_group_id(candidates['user'].values.astype(int)[train_idx][index])
        data_valid = data_all.slice(valid_idx)
        
        # print(len(candidates.user.values[valid_idx]))
        data_valid.set_group_id(candidates['user'].values.astype(int)[valid_idx])
        data_valid.set_group_weight(candidates.user.values[valid_idx])

        params = {'thread_count' : os.cpu_count(),
                  'learning_rate': learning_rate[id_type],
        **PARAMS}
        model = CatBoostRanker(**params)
        
        model.fit(data_train, eval_set = data_valid)
        
        try:
          os.mkdir(MODEL_PATH + name + VERSION)
        except:
          pass
        if fold < 5 and sample == 0 and draw_importance:
          plot_importance(X_all.columns.tolist(), model.get_feature_importance(data = data_valid, type = 'LossFunctionChange'), ax = axs[fold])
          axs[fold].bar_label(axs[fold].containers[0])
        model.save_model(MODEL_PATH + name + VERSION + f'/meowmewo_fold{fold}_sample{sample}')
  


In [17]:
for id_type, t in reversed(list(enumerate(['clicks', 'carts', 'orders']))):
  train(t, id_type, draw_importance = False)

Building pool...
Pool builed!!!!!!
FOLD: 0 Sample: 0


/home/anhphantq/.local/lib/python3.8/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'astype' of type none

File "../../../tmp/ipykernel_3022480/2935753597.py", line 35:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_3022480/2935753597.py (35)

File "../../../tmp/ipykernel_3022480/2935753597.py", line 35:
<source missing, REPL/exec in use?>

  self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_model else None)


0:	learn: 0.0000000	test: 0.9375703	best: 0.9375703 (0)	total: 1.7s	remaining: 28m 20s
1:	learn: 0.0000000	test: 0.9459590	best: 0.9459590 (1)	total: 3.27s	remaining: 27m 9s
2:	learn: 0.0000000	test: 0.9483502	best: 0.9483502 (2)	total: 4.85s	remaining: 26m 53s
3:	learn: 0.0000000	test: 0.9510412	best: 0.9510412 (3)	total: 7.02s	remaining: 29m 7s
4:	learn: 0.0000000	test: 0.9517835	best: 0.9517835 (4)	total: 9.07s	remaining: 30m 5s
5:	learn: 0.0000000	test: 0.9512183	best: 0.9517835 (4)	total: 11.2s	remaining: 31m 2s
6:	learn: 0.0000000	test: 0.9513510	best: 0.9517835 (4)	total: 13.5s	remaining: 31m 50s
7:	learn: 0.0000000	test: 0.9524848	best: 0.9524848 (7)	total: 15.9s	remaining: 32m 50s
8:	learn: 0.0000000	test: 0.9519096	best: 0.9524848 (7)	total: 18.2s	remaining: 33m 29s
9:	learn: 0.0000000	test: 0.9518570	best: 0.9524848 (7)	total: 20.5s	remaining: 33m 52s
10:	learn: 0.0000000	test: 0.9523248	best: 0.9524848 (7)	total: 22.3s	remaining: 33m 27s
11:	learn: 0.0000000	test: 0.9523879

/home/anhphantq/.local/lib/python3.8/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'astype' of type none

File "../../../tmp/ipykernel_3022480/2935753597.py", line 35:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_3022480/2935753597.py (35)

File "../../../tmp/ipykernel_3022480/2935753597.py", line 35:
<source missing, REPL/exec in use?>

  self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_model else None)


0:	learn: 0.0000000	test: 0.9426841	best: 0.9426841 (0)	total: 1.57s	remaining: 26m 10s
1:	learn: 0.0000000	test: 0.9436985	best: 0.9436985 (1)	total: 3.05s	remaining: 25m 20s
2:	learn: 0.0000000	test: 0.9474152	best: 0.9474152 (2)	total: 5.18s	remaining: 28m 41s
3:	learn: 0.0000000	test: 0.9480572	best: 0.9480572 (3)	total: 7.54s	remaining: 31m 16s
4:	learn: 0.0000000	test: 0.9498288	best: 0.9498288 (4)	total: 9.9s	remaining: 32m 49s
5:	learn: 0.0000000	test: 0.9507662	best: 0.9507662 (5)	total: 12.1s	remaining: 33m 23s
6:	learn: 0.0000000	test: 0.9511797	best: 0.9511797 (6)	total: 14.4s	remaining: 34m 1s
7:	learn: 0.0000000	test: 0.9514921	best: 0.9514921 (7)	total: 16.7s	remaining: 34m 25s
8:	learn: 0.0000000	test: 0.9518176	best: 0.9518176 (8)	total: 19s	remaining: 34m 47s
9:	learn: 0.0000000	test: 0.9520595	best: 0.9520595 (9)	total: 21.3s	remaining: 35m 7s
10:	learn: 0.0000000	test: 0.9520313	best: 0.9520595 (9)	total: 23.2s	remaining: 34m 44s
11:	learn: 0.0000000	test: 0.9524193

...

0:	learn: 0.0000000	test: 0.8585072	best: 0.8585072 (0)	total: 6.59s	remaining: 1h 49m 42s
1:	learn: 0.0000000	test: 0.8640756	best: 0.8640756 (1)	total: 9.71s	remaining: 1h 20m 47s
2:	learn: 0.0000000	test: 0.8671352	best: 0.8671352 (2)	total: 12.9s	remaining: 1h 11m 36s
3:	learn: 0.0000000	test: 0.8700754	best: 0.8700754 (3)	total: 16.2s	remaining: 1h 7m 4s
4:	learn: 0.0000000	test: 0.8715687	best: 0.8715687 (4)	total: 20s	remaining: 1h 6m 10s
5:	learn: 0.0000000	test: 0.8733806	best: 0.8733806 (5)	total: 23.3s	remaining: 1h 4m 21s
6:	learn: 0.0000000	test: 0.8745089	best: 0.8745089 (6)	total: 26.6s	remaining: 1h 2m 49s
7:	learn: 0.0000000	test: 0.8756371	best: 0.8756371 (7)	total: 30s	remaining: 1h 1m 54s
8:	learn: 0.0000000	test: 0.8760287	best: 0.8760287 (8)	total: 33.2s	remaining: 1h 53s
9:	learn: 0.0000000	test: 0.8775088	best: 0.8775088 (9)	total: 36.6s	remaining: 1h 25s
10:	learn: 0.0000000	test: 0.8776747	best: 0.8776747 (10)	total: 39.8s	remaining: 59m 41s
11:	learn: 0.00000

/home/anhphantq/.local/lib/python3.8/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'astype' of type none

File "../../../tmp/ipykernel_3022480/2935753597.py", line 35:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_3022480/2935753597.py (35)

File "../../../tmp/ipykernel_3022480/2935753597.py", line 35:
<source missing, REPL/exec in use?>

  self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_model else None)


0:	learn: 0.0000000	test: 0.8577373	best: 0.8577373 (0)	total: 5.14s	remaining: 1h 25m 33s
1:	learn: 0.0000000	test: 0.8638234	best: 0.8638234 (1)	total: 8.7s	remaining: 1h 12m 20s
2:	learn: 0.0000000	test: 0.8680843	best: 0.8680843 (2)	total: 12.2s	remaining: 1h 7m 36s
3:	learn: 0.0000000	test: 0.8694648	best: 0.8694648 (3)	total: 15.6s	remaining: 1h 4m 38s
4:	learn: 0.0000000	test: 0.8699559	best: 0.8699559 (4)	total: 18.9s	remaining: 1h 2m 31s
5:	learn: 0.0000000	test: 0.8730487	best: 0.8730487 (5)	total: 22.3s	remaining: 1h 1m 26s
6:	learn: 0.0000000	test: 0.8741770	best: 0.8741770 (6)	total: 25.6s	remaining: 1h 25s
7:	learn: 0.0000000	test: 0.8755310	best: 0.8755310 (7)	total: 28.9s	remaining: 59m 40s
8:	learn: 0.0000000	test: 0.8764800	best: 0.8764800 (8)	total: 32.8s	remaining: 1h 10s
9:	learn: 0.0000000	test: 0.8772035	best: 0.8772035 (9)	total: 37.3s	remaining: 1h 1m 33s
10:	learn: 0.0000000	test: 0.8775287	best: 0.8775287 (10)	total: 41.6s	remaining: 1h 2m 16s
11:	learn: 0.00

: 

: 